## Efficient computations of the Iwasawa $\lambda$-invariant $\lambda_p(\chi)$ for arbitary Dirichlet characters  $\chi$ using characters $\psi$ of order $p^n$.


### Preprint: *Special values of $p$-adic $L$-functions and Iwasawa λ-invariants of Dirichlet characters* by Heiko Knospe.

In [1]:
def lambda_invariant(chi,p,k):
    r"""
    Returns the lambda-invariant of the Iwasawa power series of $\chi \omega^{k}$,
    where $\chi$ is a character of the first kind and $k$ is a Teichmüller twist.
    
    First, it is checked whether $\lambda=0$. This can be done very fast
    using the first Bernoulli number of $\chi$. If $\lambda>0$ then $\chi$ is 
    multiplied with a character $\psi$ of order $p$ (i.e, of the second kind). This character is used to compute
    $\lambda$-invariants $\leq p-1$ of the original character $\chi$ and its Teichmuller twists. 
    If $\lambda \geq p$ we multiply with a character $\psi$ of higher $p$-th power order. 
    This takes more time as the conductor of $\chi \psi$ increases. However, large $\lambda$-invariants are very rare.
    
    INPUT:
    
  
    - ``chi``-- a Dirichlet character, uses the SageMath class DirichletCharacter.
    
    - ``p`` -- an odd prime number.

    - ``k`` -- a Teichmuller twist, k=0,...,p-2.
    
    
    
    OUTPUT: lambda-invariant
    
    
    """
    if p==2 or p.is_prime() == False:
        print("p must be an odd prime number.")
        return()
    
    if chi.is_primitive() == False:
        print("chi must be a primitive character.")
        return()
        
    if (chi.is_even() and k%2==1) or (chi.is_odd() and k%2==0): 
        # non-trivial p-adic L-function requires even character and even twist or odd character and odd twist
        print("The p-adic L-function is zero.")
        return()

    cond = chi.conductor() # conductor of $\chi$
    
    if gcd(cond,p*p)==p*p:
        print("chi must be a character of the first kind.")
        return() 
              
    ord = chi.order() # order of $\chi$
    
    if gcd(ord,p)>1:
        print("Not yet implemented.")
        return()

    if not (k in range(0,p-1)):
        print("k must a twist between 0 and p-2.")
        return(-1)

    if k==0:
        k=p-1 # better suited for computations
    

    F=CyclotomicField(ord*p)
    v=QQ.valuation(p).extensions(F)[0]

    # get character psi of order p and conductor p^2
    H = DirichletGroup(p*p, base_ring=CyclotomicField(p))
    for i in range(H.order()):
        if H[i].is_primitive() and H[i].order()==p:
            psi=H[i]
        exit
    
    b1=chi.bernoulli(k)
   
    
    if (chi(p)!=1 or k!=1) and v(b1)==0:
        # case rank=0, lambda=0
        
        return(0)
    else:
        # case lambda >= 1
        # check whether lambda>1
        
        G=DirichletGroup(lcm(cond,p*p),base_ring=F)
        prod=G(psi)*G(chi) # twist by character psi of order p
        b1prod=prod.bernoulli(k)
        lam= v(b1prod/k-(1-chi(p)*p^(k-1))*b1/k)*(p-1)
        if lam <= p-1:
            return(lam)
        else: 
            # case lambda>p-1
            
            # get character psi of order p^2 and conductor p^3
            H = DirichletGroup(p*p*p, base_ring=CyclotomicField(p*p))
            for i in range(H.order()):
                if H[i].is_primitive() and H[i].order()==p*p:
                    psi=H[i]
                    exit
                    
            F=CyclotomicField(ord*p*p)
            v=QQ.valuation(p).extensions(F)[0]  
            G=DirichletGroup(lcm(cond,p*p*p),base_ring=F)
            prod=G(psi)*G(chi) # twist by character psi of order p^2
            b1prod=prod.bernoulli(k)
            lam= v(b1prod/k-(1-chi(p)*p^(k-1))*b1/k)*(p-1)*p
            if lam <= (p-1)*p:
                return(lam)
            else: 
                # case lambda>(p-1)*p
                
                # get character psi of order p^3 and conductor p^4
                H = DirichletGroup(p*p*p*p, base_ring=CyclotomicField(p*p*p))
                for i in range(H.order()):
                    if H[i].is_primitive() and H[i].order()==p*p*p:
                        psi=H[i]
                        exit
                        
                F=CyclotomicField(ord*p*p*p)
                v=QQ.valuation(p).extensions(F)[0]  
                G=DirichletGroup(lcm(cond,p*p*p*p),base_ring=F)
                prod=G(psi)*G(chi) # twist by character psi of order p^3
                b1prod=prod.bernoulli(k)
                lam= v(b1prod/k-(1-chi(p)*p^(k-1))*b1/k)*(p-1)*p*p
                
                return(lam) # might be wrong for lambda>(p-1)*p^2, but this occures extremly (!) rarely.
    
    

In [2]:
# Example: conductor 19, p=7, order=6, odd character, twist=1
p=7
k=1
print(f'p={p}')
print(f'Twist={k}')
H = DirichletGroup(19, base_ring=CyclotomicField(6))
for i in range(H.order()):
    if H[i].is_primitive() and H[i].order()==6 and H[i].is_odd()==True:
        chi = H[i]
        
        print(f'{chi} ==> lambda={lambda_invariant(chi,p,1)}')
# two characters, lambda=2 and lambda=4, same result as with other methods.

p=7
Twist=1
Dirichlet character modulo 19 of conductor 19 mapping 2 |--> zeta6 ==> lambda=2
Dirichlet character modulo 19 of conductor 19 mapping 2 |--> -zeta6 + 1 ==> lambda=4


In [3]:
# Example: p=3, 700<conductor<740, order=2, twist=0, even quadratic characters
p=3
k=0
print(f'p={p}')
print(f'Twist={k}')
for n in range(701,740):
            H = DirichletGroup(n, base_ring=CyclotomicField(2))
            for i in range(H.order()):
                if H[i].order()==2 and H[i].is_even()==True and H[i].is_primitive() and gcd(H[i].conductor(),p)==1:
                    chi = H[i]
        
                    print(f'{chi} ==> lambda={lambda_invariant(chi,p,k)}')                    

p=3
Twist=0
Dirichlet character modulo 701 of conductor 701 mapping 2 |--> -1 ==> lambda=0
Dirichlet character modulo 709 of conductor 709 mapping 2 |--> -1 ==> lambda=0
Dirichlet character modulo 712 of conductor 712 mapping 535 |--> 1, 357 |--> -1, 537 |--> -1 ==> lambda=0
Dirichlet character modulo 713 of conductor 713 mapping 373 |--> -1, 530 |--> -1 ==> lambda=0
Dirichlet character modulo 716 of conductor 716 mapping 359 |--> -1, 181 |--> -1 ==> lambda=2
Dirichlet character modulo 721 of conductor 721 mapping 619 |--> -1, 211 |--> -1 ==> lambda=0
Dirichlet character modulo 728 of conductor 728 mapping 183 |--> -1, 365 |--> -1, 521 |--> -1, 561 |--> -1 ==> lambda=1
Dirichlet character modulo 733 of conductor 733 mapping 6 |--> -1 ==> lambda=3
Dirichlet character modulo 737 of conductor 737 mapping 68 |--> -1, 672 |--> -1 ==> lambda=0


In [4]:
# Example: p=5, conductor<100, order=4, twist=3, odd characters of order 4
p=5
k=3
print(f'p={p}')
print(f'Twist={k}')
for n in range(2,100):
            H = DirichletGroup(n, base_ring=CyclotomicField(4))
            for i in range(H.order()):
                if H[i].order()==4 and H[i].is_odd()==True and H[i].is_primitive() and gcd(H[i].conductor(),p)==1:
                    chi = H[i]
        
                    print(f'{chi} ==> lambda={lambda_invariant(chi,p,k)}')      

p=5
Twist=3
Dirichlet character modulo 13 of conductor 13 mapping 2 |--> zeta4 ==> lambda=0
Dirichlet character modulo 13 of conductor 13 mapping 2 |--> -zeta4 ==> lambda=0
Dirichlet character modulo 16 of conductor 16 mapping 15 |--> -1, 5 |--> zeta4 ==> lambda=0
Dirichlet character modulo 16 of conductor 16 mapping 15 |--> -1, 5 |--> -zeta4 ==> lambda=0
Dirichlet character modulo 29 of conductor 29 mapping 2 |--> zeta4 ==> lambda=0
Dirichlet character modulo 29 of conductor 29 mapping 2 |--> -zeta4 ==> lambda=0
Dirichlet character modulo 37 of conductor 37 mapping 2 |--> zeta4 ==> lambda=0
Dirichlet character modulo 37 of conductor 37 mapping 2 |--> -zeta4 ==> lambda=1
Dirichlet character modulo 48 of conductor 48 mapping 31 |--> 1, 37 |--> zeta4, 17 |--> -1 ==> lambda=1
Dirichlet character modulo 48 of conductor 48 mapping 31 |--> 1, 37 |--> -zeta4, 17 |--> -1 ==> lambda=0
Dirichlet character modulo 51 of conductor 51 mapping 35 |--> -1, 37 |--> zeta4 ==> lambda=0
Dirichlet characte

In [5]:
# Example: p=7, conductor<100, order=3, twist=0, even cubic characters
p=7
k=0
print(f'p={p}')
print(f'Twist={k}')
for n in range(2,100):
            H = DirichletGroup(n, base_ring=CyclotomicField(3))
            for i in range(H.order()):
                if H[i].order()==3 and H[i].is_even()==True and H[i].is_primitive() and gcd(H[i].conductor(),p)==1:
                    chi = H[i]
        
                    print(f'{chi} ==> lambda={lambda_invariant(chi,p,k)}')    

p=7
Twist=0
Dirichlet character modulo 9 of conductor 9 mapping 2 |--> zeta3 ==> lambda=0
Dirichlet character modulo 9 of conductor 9 mapping 2 |--> -zeta3 - 1 ==> lambda=2
Dirichlet character modulo 13 of conductor 13 mapping 2 |--> zeta3 ==> lambda=0
Dirichlet character modulo 13 of conductor 13 mapping 2 |--> -zeta3 - 1 ==> lambda=0
Dirichlet character modulo 19 of conductor 19 mapping 2 |--> zeta3 ==> lambda=0
Dirichlet character modulo 19 of conductor 19 mapping 2 |--> -zeta3 - 1 ==> lambda=0
Dirichlet character modulo 31 of conductor 31 mapping 3 |--> zeta3 ==> lambda=0
Dirichlet character modulo 31 of conductor 31 mapping 3 |--> -zeta3 - 1 ==> lambda=0
Dirichlet character modulo 37 of conductor 37 mapping 2 |--> zeta3 ==> lambda=0
Dirichlet character modulo 37 of conductor 37 mapping 2 |--> -zeta3 - 1 ==> lambda=0
Dirichlet character modulo 43 of conductor 43 mapping 3 |--> zeta3 ==> lambda=0
Dirichlet character modulo 43 of conductor 43 mapping 3 |--> -zeta3 - 1 ==> lambda=0
Di

In [6]:
# Example of a large lambda-invariant
H = DirichletGroup(71068, base_ring=CyclotomicField(2))
chi=H[7] # even quadratic character

In [7]:
p=3
k=0
print(f'p={p}')
print(f'Twist={k}')
print(f'{chi} ==> lambda={lambda_invariant(chi,p,k)}')  
# takes several minutes. lambda=11, ok!

p=3
Twist=0
Dirichlet character modulo 71068 of conductor 71068 mapping 35535 |--> -1, 33905 |--> -1, 53629 |--> -1 ==> lambda=11
